In [248]:
#import stuff
import numpy as np
import sklearn as sk
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [249]:
#import cleaned up datasets
protein_df = pd.read_csv("data/data_clean/protein_data_clean.csv")
kg_df = pd.read_csv("data/data_clean/kg_data_clean.csv")
kcal_df = pd.read_csv("data/data_clean/kcal_data_clean.csv")
fat_df = pd.read_csv("data/data_clean/fat_data_clean.csv")

In [250]:
#To aid regression, drop features that probably highly correlate with other features
#We have absolutely no need for obesity and undernourishment
protein_df = protein_df.drop(['Obesity','Undernourished'],axis=1)
kg_df = kg_df.drop(['Obesity','Undernourished'],axis=1)
kcal_df = kcal_df.drop(['Obesity','Undernourished'],axis=1)
fat_df = fat_df.drop(['Obesity','Undernourished'],axis=1)
#protein_df = protein_df.drop(['Obesity','Animal fats_p','Aquatic Products, Other_p', 'Sugar Crops_p', 'Vegetal Products_p', 'Vegetable Oils_p', 'Miscellaneous_p'],axis=1)
#kg_df = kg_df.drop(['Obesity','Animal fats_kg','Aquatic Products, Other_kg', 'Sugar Crops_kg', 'Vegetal Products_kg', 'Vegetable Oils_kg', 'Miscellaneous_kg'],axis=1)
#kcal_df = kcal_df.drop(['Obesity','Animal fats_kcal','Aquatic Products, Other_kcal', 'Sugar Crops_kcal', 'Vegetal Products_kcal', 'Vegetable Oils_kcal', 'Miscellaneous_kcal'],axis=1)
#fat_df = fat_df.drop(['Obesity','Animal fats_f','Aquatic Products, Other_f', 'Sugar Crops_f', 'Vegetal Products_f', 'Vegetable Oils_f', 'Miscellaneous_f'],axis=1)

<h3>
I want to figure out what the lambda best value to penalize our model is for LASSO, for each dataset set.
I will also print out the weight vector, so we can determine what features LASSO determines are actually important.
</h3>

In [251]:
#gonna try different scoring metrics
def loop_lasso_mse(X_train, y_train, x):
    best_a = 0.001
    best_score = np.inf
    test_a_values = np.arange(0.001,0.1,0.001)
    for a in range(0,len(test_a_values)):
        Lreg = linear_model.Lasso(alpha = test_a_values[a], tol=0.1)
        Lreg.fit(X_train, y_train)
        score = cross_val_score(Lreg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        if(np.average(score)*-1<best_score):
            best_score=np.average(score)*-1
            best_a = test_a_values[a]
    print("Best score :",best_score)
    print("Best Alpha :",best_a)
    #Get the best model's coefs
    Lreg = linear_model.Lasso(alpha = best_a)
    Lreg.fit(X_train, y_train)
    print("Weight coefficients:")
    for c in range(1,len(Lreg.coef_)):
        if(Lreg.coef_[c]>0.0):
            print(x.columns[c],":",Lreg.coef_[c])
    print('\n')

In [252]:
def loop_lasso_rmse(X_train, y_train, x):
    best_a = 0.001
    best_score = np.inf
    test_a_values = np.arange(0.001,0.1,0.001)
    for a in range(0,len(test_a_values)):
        Lreg = linear_model.Lasso(alpha = test_a_values[a], tol=0.1)
        Lreg.fit(X_train, y_train)
        score = cross_val_score(Lreg, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
        if(np.average(score)*-1<best_score):
            best_score=np.average(score)*-1
            best_a = test_a_values[a]
    print("Best score :",best_score)
    print("Best Alpha :",best_a)
    #Get the best model's coefs
    Lreg = linear_model.Lasso(alpha = best_a)
    Lreg.fit(X_train, y_train)
    print("Weight coefficients:")
    for c in range(1,len(Lreg.coef_)):
        if(Lreg.coef_[c]>0.0):
            print(x.columns[c],":",Lreg.coef_[c])
    print('\n')

In [255]:
def LASSO(df):
    df_x = df.drop(['Country', 'Population','Active','Recovered','Deaths','Confirmed'],axis=1)
    #to help regression, drop some features that are probably highly correlated
    #scale the data
    x = MinMaxScaler().fit_transform(df_x)

    #confirmed
    print("Confirmed:")
    confirmed_index = -1
    for c in range(0,len(df.columns)):
        if(df.columns[c] == "Confirmed"):
            confirmed_index = c
    y = df.iloc[:,confirmed_index].values
    print("RMSE")
    loop_lasso_rmse(x,y, df_x)
    print("MSE")
    loop_lasso_mse(x,y, df_x)
    
    #deaths
    print("Deaths:")
    deaths_index = -1
    for c in range(0,len(df.columns)):
        if(df.columns[c] == "Deaths"):
            deaths_index = c
    y = df.iloc[:,deaths_index].values*100
    print("RMSE")
    loop_lasso_rmse(x,y, df_x)
    print("MSE")
    loop_lasso_mse(x,y, df_x)
    
    #recovered
    print("Recovered:")
    recovered_index = -1
    for c in range(0,len(df.columns)):
        if(df.columns[c] == "Recovered"):
            recovered_index = c
    y = df.iloc[:,recovered_index].values
    print("RMSE")
    loop_lasso_rmse(x,y, df_x)
    print("MSE")
    loop_lasso_mse(x,y, df_x)

In [256]:
LASSO(protein_df)

Confirmed:
RMSE
Best score : 1.7176617133076484
Best Alpha : 0.010000000000000002
Weight coefficients:
Animal fats_p : 0.18946673932943206
Eggs_p : 1.3425266696369968
Meat_p : 1.3166610363559794
Milk - Excluding Butter_p : 3.602900093708701
Stimulants_p : 1.4372746722711465
Sugar & Sweeteners_p : 0.11049615850011689
Treenuts_p : 1.3714115177145116


MSE
Best score : 3.013515451884288
Best Alpha : 0.010000000000000002
Weight coefficients:
Animal fats_p : 0.18946673932943206
Eggs_p : 1.3425266696369968
Meat_p : 1.3166610363559794
Milk - Excluding Butter_p : 3.602900093708701
Stimulants_p : 1.4372746722711465
Sugar & Sweeteners_p : 0.11049615850011689
Treenuts_p : 1.3714115177145116


Deaths:
RMSE
Best score : 3.69111966225122
Best Alpha : 0.024
Weight coefficients:
Animal fats_p : 2.8314872404466014
Eggs_p : 3.7219916979875807
Meat_p : 3.020690270053019
Milk - Excluding Butter_p : 4.238174953065353
Stimulants_p : 0.7126778014694737
Sugar & Sweeteners_p : 1.7322147040192228
Treenuts_p : 0

In [257]:
LASSO(fat_df)

Confirmed:
RMSE
Best score : 1.7827436864828112
Best Alpha : 0.007
Weight coefficients:
Animal fats_f : 2.3848576769578305
Eggs_f : 0.931518225938917
Milk - Excluding Butter_f : 1.0106988290638603
Stimulants_f : 3.5000778951038205
Treenuts_f : 1.0318898677971675


MSE
Best score : 3.2373745184803995
Best Alpha : 0.007
Weight coefficients:
Animal fats_f : 2.3848576769578305
Eggs_f : 0.931518225938917
Milk - Excluding Butter_f : 1.0106988290638603
Stimulants_f : 3.5000778951038205
Treenuts_f : 1.0318898677971675


Deaths:
RMSE
Best score : 4.005846664504503
Best Alpha : 0.046
Weight coefficients:
Animal fats_f : 6.859203023221479
Eggs_f : 2.149432824666097
Milk - Excluding Butter_f : 1.1328317199087565
Stimulants_f : 2.556563507174045


MSE
Best score : 16.186459673359018
Best Alpha : 0.046
Weight coefficients:
Animal fats_f : 6.859203023221479
Eggs_f : 2.149432824666097
Milk - Excluding Butter_f : 1.1328317199087565
Stimulants_f : 2.556563507174045


Recovered:
RMSE
Best score : 1.52736

In [258]:
LASSO(kg_df)

Confirmed:
RMSE
Best score : 1.8370941105948926
Best Alpha : 0.010000000000000002
Weight coefficients:
Animal fats_kg : 0.6827690731905616
Eggs_kg : 0.8437262233037577
Milk - Excluding Butter_kg : 3.0892996848242293
Stimulants_kg : 0.5369614480020173
Sugar & Sweeteners_kg : 1.6559106609622452
Treenuts_kg : 1.3099565419546302
Vegetables_kg : 0.244838562514475


MSE
Best score : 3.410092224817679
Best Alpha : 0.010000000000000002
Weight coefficients:
Animal fats_kg : 0.6827690731905616
Eggs_kg : 0.8437262233037577
Milk - Excluding Butter_kg : 3.0892996848242293
Stimulants_kg : 0.5369614480020173
Sugar & Sweeteners_kg : 1.6559106609622452
Treenuts_kg : 1.3099565419546302
Vegetables_kg : 0.244838562514475


Deaths:
RMSE
Best score : 3.772119512643262
Best Alpha : 0.037000000000000005
Weight coefficients:
Animal fats_kg : 2.6788345121016452
Eggs_kg : 3.04208785472695
Fruits - Excluding Wine_kg : 0.23631374856921844
Milk - Excluding Butter_kg : 4.569478817733755
Treenuts_kg : 1.8968704294651

In [259]:
LASSO(kcal_df)

Confirmed:
RMSE
Best score : 1.7136492037415976
Best Alpha : 0.014000000000000002
Weight coefficients:
Animal fats_kcal : 1.3421664298467444
Eggs_kcal : 1.20935697846046
Fruits - Excluding Wine_kcal : 0.2551125138060378
Milk - Excluding Butter_kcal : 1.9912362724150643
Stimulants_kcal : 3.7453082760314316
Sugar & Sweeteners_kcal : 0.7074807460042866
Treenuts_kcal : 1.4100752747986813
Vegetable Oils_kcal : 0.5809534203000907


MSE
Best score : 2.985037469369159
Best Alpha : 0.014000000000000002
Weight coefficients:
Animal fats_kcal : 1.3421664298467444
Eggs_kcal : 1.20935697846046
Fruits - Excluding Wine_kcal : 0.2551125138060378
Milk - Excluding Butter_kcal : 1.9912362724150643
Stimulants_kcal : 3.7453082760314316
Sugar & Sweeteners_kcal : 0.7074807460042866
Treenuts_kcal : 1.4100752747986813
Vegetable Oils_kcal : 0.5809534203000907


Deaths:
RMSE
Best score : 3.8336473306894376
Best Alpha : 0.045
Weight coefficients:
Animal fats_kcal : 5.2491882279553
Eggs_kcal : 3.171556464227425
Mil